In [17]:
%%capture
!pip install ipython-sql;

%load_ext sql
%sql sqlite:///mydatabase.db

In [18]:
%sql SELECT First_Name, Last_Name,Email FROM Client

 * sqlite:///mydatabase.db
Done.


First_Name,Last_Name,Email
John,Doe,john.doe@email.com
Jane,Smith,jane.smith@email.com
Emily,Johnson,emily.johnson@email.com


In [19]:
%%sql
SELECT T.Transaction_ID, P.Product_Name, C.First_Name, C.Last_Name
FROM Transactions T
INNER JOIN Product P ON T.Product_ID = P.Product_ID
INNER JOIN Client C ON T.Client_ID = C.Client_ID;

 * sqlite:///mydatabase.db
Done.


Transaction_ID,Product_Name,First_Name,Last_Name
1,Credit Card,John,Doe
2,Savings Account,Jane,Smith
3,Personal Loan,Emily,Johnson


In [20]:
%%sql
SELECT
    Client_ID,
    SUM(Loan_Amount) AS Total_Loan_Amount,
    AVG(Interest_Rate) AS Average_Interest_Rate
FROM Loan
GROUP BY Client_ID;

 * sqlite:///mydatabase.db
Done.


Client_ID,Total_Loan_Amount,Average_Interest_Rate
1,10000,5.0
2,20000,6.0
3,30000,7.0


In [21]:
%%sql
SELECT
    Client_ID,
    Transaction_Date,
    Amount,
    SUM(Amount) OVER (PARTITION BY Client_ID ORDER BY Transaction_Date) AS Cumulative_Total
FROM Transactions
ORDER BY Client_ID, Transaction_Date;

 * sqlite:///mydatabase.db
Done.


Client_ID,Transaction_Date,Amount,Cumulative_Total
1,2023-01-01,1000,1000
2,2023-01-02,2000,2000
3,2023-01-03,3000,3000


In [23]:
%%sql
WITH ClientLiabilities AS (
    SELECT
        C.Client_ID,
        COALESCE(B.Balance, 0) AS Balance,
        COALESCE(L.Loan_Amount, 0) AS Loan_Amount,
        COALESCE(B.Balance, 0) + COALESCE(L.Loan_Amount, 0) AS Total_Liability
    FROM Client C
    LEFT JOIN Balance B ON C.Client_ID = B.Client_ID
    LEFT JOIN Loan L ON C.Client_ID = L.Client_ID
)
SELECT
    CL.Client_ID,
    CL.Balance,
    CL.Loan_Amount,
    CL.Total_Liability
FROM ClientLiabilities CL
WHERE CL.Balance > 0 AND CL.Loan_Amount > 0
ORDER BY CL.Total_Liability DESC;

 * sqlite:///mydatabase.db
Done.


Client_ID,Balance,Loan_Amount,Total_Liability
3,3000,30000,33000
2,2000,20000,22000
1,1000,10000,11000
